# Piqasso Part II: MSD State Encoding & Steane QEC

**QuEra iQuHack 2026 — Step 2**

This notebook implements:
1. **MSD State Encoding circuit** — Encode a state into the [[7,1,3]] (Steane/color) code, following QuEra's magic-state distillation work.
2. **Steane QEC** — One round of syndrome extraction: measure the six stabilizers of the [[7,1,3]] code and read out the syndrome.

References: [QuEra MSD (2412.15165)](https://arxiv.org/abs/2412.15165), [Steane QEC (2312.09745)](https://arxiv.org/pdf/2312.09745).

## 1. Setup

Install dependencies (run once; on Colab: **Runtime → Restart session** after). Then import Bloqade (Squin), Cirq interop, and Stim.

In [1]:
!pip install -q numpy cirq "bloqade-circuit[cirq,stim,tsim]>=0.11.0"
print("Done. On Colab: Runtime -> Restart session, then run the rest.")

Done. On Colab: Runtime -> Restart session, then run the rest.


  error: subprocess-exited-with-error
  
  × Building wheel for stim (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [20 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating python_build_stim\lib.win-amd64-cpython-313\stim
      copying glue\python\src\stim\_main_argv.py -> python_build_stim\lib.win-amd64-cpython-313\stim
      copying glue\python\src\stim\__init__.py -> python_build_stim\lib.win-amd64-cpython-313\stim
      running egg_info
      writing stim.egg-info\PKG-INFO
      writing dependency_links to stim.egg-info\dependency_links.txt
      writing entry points to stim.egg-info\entry_points.txt
      writing requirements to stim.egg-info\requires.txt
      writing top-level names to stim.egg-info\top_level.txt
      reading manifest file 'stim.egg-info\SOURCES.txt'
      reading manifest template 'MANIFEST.in'
      adding license file 'LICENSE'
      writing manifest file 'stim.egg-info\SOURCES.txt'
      cop

In [1]:
from typing import Literal
import numpy as np
from bloqade import squin
from bloqade.types import Qubit
from kirin.dialects import ilist
from math import pi
import bloqade.stim
import bloqade.tsim
from bloqade.cirq_utils import load_circuit
from bloqade.cirq_utils.emit import emit_circuit

## 2. Magic State Preparation and Injection

Prepare the MSD circuit by introducing a magic state and injecting it into the 7 qubit code

In [ ]:
@squin.kernel
def preparemagicstate(qubit):
    squin.i(qubit)


@squin.kernel
def injection(q: ilist.IList[Qubit, Literal[7]]):
    """Apply magic-state injection to the 7-qubit register `q` (allocated by caller)."""
    preparemagicstate(q[6])
    for i in range(6):
        squin.ry(-pi / 2, q[i])
    squin.cz(q[1], q[2])
    squin.cz(q[3], q[4])
    squin.cz(q[5], q[6])
    squin.ry(pi / 2, q[6])
    squin.cz(q[0], q[3])
    squin.cz(q[2], q[5])
    squin.cz(q[4], q[6])
    for i in range(2, 7):
        squin.ry(pi / 2, q[i])
    for i in range(0, 5, 2):
        squin.cz(q[i], q[i + 1])
    squin.ry(pi / 2, q[1])
    squin.ry(pi / 2, q[2])
    squin.ry(pi / 2, q[4])

@squin.kernel
def injection_only():
    """Wrapper: allocate 7 qubits and run injection (for emit_circuit without args)."""
    q = squin.qalloc(7)
    injection(q)

and let's sanity check our circuit

In [3]:
# Injection only (7 qubits) — use wrapper kernel since injection(q) takes a parameter
cirq_diagram = emit_circuit(injection_only)
squin_for_diagram = load_circuit(cirq_diagram)
tsim_circ = bloqade.tsim.Circuit(squin_for_diagram)
fig = tsim_circ.diagram(height=400)
fig

## 3. Extraction

Let's now move on to extracting our codes. 

In [10]:
@squin.kernel
def encode_plus(q: ilist.IList[Qubit, Literal[7]]):
    for i in range(7):
        squin.h(q[i])

@squin.kernel
def encode_0(q: ilist.IList[Qubit, Literal[7]]):
    for i in range(7):
        squin.i(q[i])

@squin.kernel
def a3extraction(data: ilist.IList[Qubit, Literal[7]],ancilla: ilist.IList[Qubit, Literal[7]]):
    """Full fault-tolerant Steane EC cycle: 14 qubits (7 data + 7 ancilla). First half: ancilla |+⟩_L, CNOT data→ancilla, measure (X-syndrome), reset. Second half: ancilla |0⟩_L, CNOT ancilla→data, H on ancilla, measure (Z-syndrome)."""
    encode_plus(ancilla)
    for i in range(7):
        squin.cx(data[i], ancilla[i])
    squin.broadcast.measure(ancilla)
    squin.broadcast.reset(ancilla)
    encode_0(ancilla)
    for i in range(7):
        squin.cx(ancilla[i], data[i])
    for i in range(7):
        squin.h(ancilla[i])
    squin.broadcast.measure(ancilla)

@squin.kernel
def execution():
    """Sequential: (1) inject magic state into 7 data qubits; (2) run a3 extraction on that data + 7 ancillas."""
    q = squin.qalloc(7)
    injection(q)
    ancilla = squin.qalloc(7)
    a3extraction(q, ancilla)

**Note on full MSD state injection:** The QuEra color-code figure uses controlled-√Y gates to encode an arbitrary |Ψ(θ,φ)⟩. That circuit is non-Clifford; to simulate it you would use **Tsim** (or a state-vector simulator) with the appropriate gates. The Clifford encoding above is sufficient to test the **Steane QEC** pipeline (encoding → syndrome extraction) with Stim.

In [11]:
# Emit a3extraction (14 qubits) and display diagram

cirq_a3 = emit_circuit(execution)
squin_a3 = load_circuit(cirq_a3)
tsim_a3 = bloqade.tsim.Circuit(squin_a3)
fig_a3 = tsim_a3.diagram(height=500)
fig_a3

InterpreterError: SSAValue <ResultValue[Qubit] qubit, uses: 1> not found